# Exploratory Data Analysis (EDA)

### Retrieve data

In [ ]:
!pip install gdown

In [ ]:
!gdown "1KZTqa1PHu-FFtwE9PBBoF1jy-3_iUy2b"

Downloading...
From: https://drive.google.com/uc?id=1KZTqa1PHu-FFtwE9PBBoF1jy-3_iUy2b
To: /content/digit-recognizer.zip
100% 16.1M/16.1M [00:00<00:00, 71.1MB/s]


In [ ]:
!unzip "digit-recognizer.zip" && rm "digit-recognizer.zip"

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!ls

sample_data  sample_submission.csv  test.csv  train.csv


# Model Experiments

# Results Analysis